In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.model_selection import train_test_split
from collections import Counter

## 1. 데이터 불러오기 (2021 기준)
    - 1-1. 휴장일
    - 1-2. Newsfile 
    - 1-3. 반도체(005290) : 상승률 1.5 이상--1, 상승률 1.5 이하--0 라벨링 데이터

### 1-1. 2021 휴장일 날짜

In [2]:
closedate_2021 = pd.read_excel('../data/closedate_2021.xls')
closedate_2021.head(3)

,일자 및 요일,요일구분,비고
0,2021-01-01,금요일,신정
1,2021-02-11,목요일,설날
2,2021-02-12,금요일,설날


In [3]:
closedate_2021 = closedate_2021[['일자 및 요일', '요일구분']]
closedate_2021.head(3)

,일자 및 요일,요일구분
0,2021-01-01,금요일
1,2021-02-11,목요일
2,2021-02-12,금요일


### 1-2. newsfile

- train_data

In [27]:
news_train = pd.read_csv('../data/train_data_hanja_translate.csv')
news_train.head(3)

,Unnamed: 0,date,title,nouns
0,37650,20190109,작년 분기 어닝 쇼크 삼성전자글로벌 반도체 왕좌 다시 내주나,작년 분기 어닝 쇼크 삼성 전자 글로벌 반도체 왕좌 다시 내주
1,9503,20210714,삼성전자카이스트 반도체 계약학과 신설인재 양성 가속,삼성 전자 카이스트 반도체 계약 학과 신설 인재 양성 가속
2,29948,20190806,사진반도체 공장 둘러보는 이재용 부회장,사진 반도체 공장 이재용 부회장


In [28]:
news_train = news_train[['date', 'title', 'nouns']]
news_train.head(3)

,date,title,nouns
0,20190109,작년 분기 어닝 쇼크 삼성전자글로벌 반도체 왕좌 다시 내주나,작년 분기 어닝 쇼크 삼성 전자 글로벌 반도체 왕좌 다시 내주
1,20210714,삼성전자카이스트 반도체 계약학과 신설인재 양성 가속,삼성 전자 카이스트 반도체 계약 학과 신설 인재 양성 가속
2,20190806,사진반도체 공장 둘러보는 이재용 부회장,사진 반도체 공장 이재용 부회장


- test_data

In [29]:
news_test = pd.read_csv('../data/test_data_hanja_translate.csv')
news_test.head(3)

,Unnamed: 0,date,title,nouns
0,28288,20191008,리탈 제회 반도체대전SEDEX 참가,리탈 회 반도체 대전 참가
1,3825,20211130,정부 반도체용 수입 산업가스 수급 안정화고압가스 반송기한 최대 년으로 연,정부 반도체 용 수입 산업 가스 수급 안정화 고압 가스 반송 기한 최대 년 연
2,11223,20210524,김대호의 경제읽기 굳건해진 반도체배터리 협력경제동맹 새 지평,김대호 경제 해진 반도체 배터리 협력 경제 동맹 새 지평


In [30]:
news_test = news_test[['date', 'title', 'nouns']]
news_test.head(3)

,date,title,nouns
0,20191008,리탈 제회 반도체대전SEDEX 참가,리탈 회 반도체 대전 참가
1,20211130,정부 반도체용 수입 산업가스 수급 안정화고압가스 반송기한 최대 년으로 연,정부 반도체 용 수입 산업 가스 수급 안정화 고압 가스 반송 기한 최대 년 연
2,20210524,김대호의 경제읽기 굳건해진 반도체배터리 협력경제동맹 새 지평,김대호 경제 해진 반도체 배터리 협력 경제 동맹 새 지평


- concat

In [32]:
news_data = pd.concat([news_train, news_test])
news_data.head(10)

,date,title,nouns
0,20190109,작년 분기 어닝 쇼크 삼성전자글로벌 반도체 왕좌 다시 내주나,작년 분기 어닝 쇼크 삼성 전자 글로벌 반도체 왕좌 다시 내주
1,20210714,삼성전자카이스트 반도체 계약학과 신설인재 양성 가속,삼성 전자 카이스트 반도체 계약 학과 신설 인재 양성 가속
2,20190806,사진반도체 공장 둘러보는 이재용 부회장,사진 반도체 공장 이재용 부회장
3,20200728,코로나에 월 수출물량 감소 반도체 두달 연속 증가,코로나 월 수출 물량 감소 반도체 달 연속 증가
4,20210531,단독 현대차 토종 팹리스와 차반도체 국산화 나선다,단독 현대차 토종 팹리스 차 반도체 국산
5,20210419,삼성전자 조 역대급 투자 기대반도체 장비주 급등,삼성 전자 조 역대 투자 반도체 장비 주 급등
6,20181226,지자체발 반도체 대전 이천시용인시의회 SK하이닉스 잡기 나섰다,지자체 발 반도체 대전 이천시 용인시 의회 하이닉스 잡기
7,20210830,반도체 공정때 나오는 폐산 재활용수입대체 효과,반도체 공정 때 폐산 재활용 수입 대체 효과
8,20200909,기 신도시 사전청약화웨이에 반도체 공급 중단항체치료제 이번달 생산,기 신도시 청약 화웨이 반도체 공급 중단 항체 치료 제 이번 달 생산
9,20180320,빅데이터로 불량률 삼성 완벽한 반도체 미션,빅데이터 불량 률 삼성 반도체 미션


- 날짜순 정렬

In [18]:
news_data = news_data.sort_values(by="date")
news_data.head(10)

,date,title,nouns
19650,20151019,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
3420,20151019,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
1798,20151019,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상
34859,20151019,제주반도체 해외투자유치설 조회공시 요구,제주 반도체 해외 투자 유치 설 조회 공시 요구
9280,20151019,반도체사 인수합병 봇물올 들어 억달러 돌파,반도 체사 인수합병 봇 물올 달러 돌파
24071,20151019,세계 반도체 MA 억달러,세계 반도체 달러
12374,20151020,STS반도체 임시주총서 사내이사 선임건 의결,반도체 총서 사내 이사 선임 건 의결
8164,20151020,제주반도체 램스웨이와 만불 투자 MOU 체결,제주 반도체 램 스웨 불 투자 체결
13915,20151020,STS반도체 김영민 대표이사 신규 선임,반도체 김영민 대표이사 신규 선임
41391,20151020,제주반도체 종속회사 램스웨이 만달러 투자 유치,제주 반도체 종 회사 램 스웨 달러 투자 유치


In [41]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 14610
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
 2   nouns   58442 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.8+ MB


In [44]:
news_data = news_data.astype({'date':'string'})
news_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 14610
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  string
 1   title   58442 non-null  object
 2   nouns   58442 non-null  object
dtypes: object(2), string(1)
memory usage: 3.8+ MB


- 2021년도 date

In [52]:
news_data_2021 = news_data[news_data['date'].str.contains('2021')]
news_data_2021

,date,title,nouns
1,20210714,삼성전자카이스트 반도체 계약학과 신설인재 양성 가속,삼성 전자 카이스트 반도체 계약 학과 신설 인재 양성 가속
4,20210531,단독 현대차 토종 팹리스와 차반도체 국산화 나선다,단독 현대차 토종 팹리스 차 반도체 국산
5,20210419,삼성전자 조 역대급 투자 기대반도체 장비주 급등,삼성 전자 조 역대 투자 반도체 장비 주 급등
7,20210830,반도체 공정때 나오는 폐산 재활용수입대체 효과,반도체 공정 때 폐산 재활용 수입 대체 효과
13,20210720,전세계 반도체 기업 목줄 잡은 ASML년 새 매출 두 배로,전세계 반도체 기업 목줄 년 새 매출 두 배
...,...,...,...
14578,20211101,보반도체철강 등 개 품목 수출 강세자동차는 주춤,보 반도체 철강 등 개 품목 수출 강세 자동차 주춤
14598,20210514,K반도체 육성반도체 소부장주 주목,반도체 육성 반도체 소부 장주 주목
14599,20210506,현대차 공장 찾은 송영길 차량 반도체 대책 뒷받침,현대차 공장 송영길 차량 반도체 대책 뒷받침
14603,20210913,중제조업경기 둔화 한수출에 부정적반도체 모멘텀 우려,중제 조업 경기 둔화 수출 부정 반도체 모멘텀 우려


## 1-3. 반도체(005290) 상승률 기준 라벨링 데이터

In [56]:
updown_2021 = pd.read_excel('../data/반도체주가데이터/반도체주가데이터/동진쎄미캠(005290).xlsx')
updown_2021.head(3)

,Unnamed: 0.1,Unnamed: 0,일자,현재가,당일종가-어제종가,상승률,label
0,0,0,20220314,31900,-1500,-0.044910,0
1,1,1,20220311,33400,-300,-0.008902,0
2,2,2,20220310,33700,550,0.016591,1


In [57]:
updown_2021.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
updown_2021.head(3)

,일자,현재가,당일종가-어제종가,상승률,label
0,20220314,31900,-1500,-0.044910,0
1,20220311,33400,-300,-0.008902,0
2,20220310,33700,550,0.016591,1
